# EDA: Exploratory Data Analysis

## 1. Setup + Imports

In [40]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

ROOT = Path.cwd().parent

DATA = ROOT / "data" / "processed" / "analytics_table.parquet"
FIGS = ROOT / "reports" / "figures"
FIGS.mkdir(parents=True,exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

## 2. Load processed data

In [41]:

df = pd.read_parquet(DATA)
print("rows:", len(df), "cols:", len(df.columns))

rows: 5 cols: 18


## 3. Quick audit

In [42]:
print(df.dtypes.head(15))
missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)

order_id              string[python]
user_id               string[python]
amount                       Float64
quantity                       Int64
created_at       datetime64[ns, UTC]
status                        object
status_clean                  object
amount_isna                     bool
quantity_isna                   bool
date                          object
year                         float64
month                 string[python]
dow                           object
hour                         float64
country                       object
dtype: object
quantity              1
amount                1
amount _is_outlier    1
amount_winsor         1
created_at            1
date                  1
hour                  1
dow                   1
month                 1
year                  1
dtype: int64


## 4. Questions + results

##### Q1: Which countries generate the highest total revenue?

##### Q2: How does revenue evolve over time on a monthly basis?

##### Q3: What does a typical order amount look like after winsorization?

#### Question 1 — Revenue by Country:


In [43]:
rev = (
df.groupby("country", dropna=False)
.agg(
n=("order_id", "size"),
revenue=("amount", "sum"),
aov=("amount", "mean"),
)
.reset_index()
.sort_values("revenue", ascending=False)
)
fig = px.bar(
rev,
x="country",
y="revenue",
title="Revenue by Country (All Data)",
)
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
save_fig(fig, FIGS / "revenue_by_country.png")
fig

#### Question 2 — Revenue Trend (Monthly)

In [44]:

df["signup_date"] = pd.to_datetime(
    df["signup_date"],
    dayfirst=True,
    errors="coerce"
)


df["day_of_week"] = df["signup_date"].dt.day_name()

dow_order = [
   "Saturday","Sunday", "Monday", "Tuesday", "Wednesday",
    "Thursday", "Friday"
]

df["day_of_week"] = pd.Categorical(
    df["day_of_week"],
    categories=dow_order,
    ordered=True
)


trend = (
    df.groupby("day_of_week", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum"),
    )
    .reset_index()
    .sort_values("day_of_week")
)

trend

fig = px.line(
    trend,
    x="day_of_week",
    y="revenue",
    title="Revenue by Day of Week (Signup Date)",
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Day of Week")
fig.update_yaxes(title_text="Revenue")

save_fig(fig, FIGS / "revenue_trend_day_of_week.png")
fig

C:\Users\اشواق\AppData\Local\Temp\ipykernel_11428\3377794510.py:1: UserWarning:

Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.

C:\Users\اشواق\AppData\Local\Temp\ipykernel_11428\3377794510.py:23: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



#### Question 3: Amount distribution (winsorized)

In [45]:
fig = px.histogram(
df,
x="amount_winsor",
nbins=30,
title="Order Amount Distribution (Winsorized)",
)
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of Orders")
save_fig(fig, FIGS / "amount_hist_winsor.png")
fig

## 5. Bootstrap comparison

In [46]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"


    diffs = []
    for _ in range(n_boot):
            sa = rng.choice(a, size=len(a), replace=True)
            sb = rng.choice(b, size=len(b), replace=True)
            diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)
    return {
            "diff_mean": float(a.mean() - b.mean()),
            "ci_low": float(np.quantile(diffs, 0.025)),
            "ci_high": float(np.quantile(diffs, 0.975)),
        }
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))
a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]
print("n_SA:", len(a), "n_AE:", len(b))
bootstrap_diff_means(a, b, n_boot=2000, seed=0)

n_SA: 4 n_AE: 1


{'diff_mean': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}

## 6. Findings + caveats

Key findings:

Revenue concentration varies strongly by country.

Temporal patterns suggest seasonality or campaign effects.

Typical order values are stable once extreme outliers are controlled.

Caveats:

Observational data; no causal claims.

Results depend on processed definitions (winsorization, cleaning rules).

Country-level comparisons ignore customer mix and operational differences.